In [3]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.7 MB/s eta 0:00:00


In [1]:
!pip install torch torchvision torchaudio
!pip install git+https://github.com/ultralytics/yolov5.git


  Cloning https://github.com/ultralytics/yolov5.git to /tmp/pip-req-build-mwo1edi2
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/yolov5.git /tmp/pip-req-build-mwo1edi2
  Resolved https://github.com/ultralytics/yolov5.git to commit 907bef2f63771a352346d0b4f068946d7f961f0c
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
import cv2  # مكتبة للتعامل مع الفيديو والصور
import torch  # مكتبة للتعلم العميق
from PIL import Image, ImageDraw  # مكتبة لتحرير الصور
import gradio as gr  # مكتبة لإنشاء واجهات تفاعلية
import pandas as pd  # مكتبة للتعامل مع البيانات
from transformers import pipeline  # مكتبة للترجمة

# تحميل نموذج YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# تحميل نموذج الترجمة
translator = pipeline("translation_en_to_ar", model="Helsinki-NLP/opus-mt-en-ar")

# دالة لاكتشاف الكائنات ورسم الإطارات في الصور
def detect_and_draw_image(input_image):
    results = model(input_image)
    detections = results.xyxy[0].numpy()

    draw = ImageDraw.Draw(input_image)

    counts = {}
    for detection in detections:
        xmin, ymin, xmax, ymax, conf, class_id = detection
        label = model.names[int(class_id)]
        counts[label] = counts.get(label, 0) + 1

        draw.rectangle([(xmin, ymin), (xmax, ymax)], outline="red", width=2)
        draw.text((xmin, ymin), f"{label}: {conf:.2f}", fill="white")

    # ترجمة التسميات إلى العربية
    translated_labels = translator(list(counts.keys()))

    # إنشاء DataFrame لتخزين النتائج
    df = pd.DataFrame({
        'Label (English)': list(counts.keys()),
        'Label (Arabic)': [t['translation_text'] for t in translated_labels],
        'Object Count': list(counts.values())
    })

    return input_image, df

# دالة لاكتشاف الكائنات ورسم الإطارات في الفيديو
def detect_and_draw_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    overall_counts = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (640, 480))
        results = model(frame)
        detections = results.xyxy[0].numpy()

        for detection in detections:
            xmin, ymin, xmax, ymax, conf, class_id = detection
            label = model.names[int(class_id)]
            overall_counts[label] = overall_counts.get(label, 0) + 1

            cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
            cv2.putText(frame, f"{label}: {conf:.2f}", (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        frames.append(frame)

    cap.release()

    output_path = 'output.mp4'
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (640, 480))

    for frame in frames:
        out.write(frame)
    out.release()

    # ترجمة التسميات إلى العربية
    translated_labels = translator(list(overall_counts.keys()))

    # إنشاء DataFrame لتخزين النتائج
    df = pd.DataFrame({
        'Label (English)': list(overall_counts.keys()),
        'Label (Arabic)': [t['translation_text'] for t in translated_labels],
        'Object Count': list(overall_counts.values())
    })

    return output_path, df

# إنشاء واجهات لتطبيق Gradio
image_interface = gr.Interface(
    fn=detect_and_draw_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=[gr.Image(type="pil"), gr.Dataframe(label="Object Counts")],
    title="Object Detection for Images",
    description="Upload an image to see the objects detected by YOLOv5 with bounding boxes and their counts."
)

video_interface = gr.Interface(
    fn=detect_and_draw_video,
    inputs=gr.Video(label="Upload Video"),
    outputs=[gr.Video(label="Processed Video"), gr.Dataframe(label="Object Counts")],
    title="Object Detection for Videos",
    description="Upload a video to see the objects detected by YOLOv5 with bounding boxes and their counts."
)

# دمج الواجهات في تطبيق واحد
app = gr.TabbedInterface([image_interface, video_interface], ["Image Detection", "Video Detection"])

# تشغيل التطبيق
app.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
